# Tutorial on how to fit emission line and use IFU data - KMOS/SINFONI/JWST etc

Here I am going to go through basic of analysing IFU data and fitting emission lines. First, in order to quickly look at IFU cubes, I recommend QFits. https://www.mpe.mpg.de/~ott/QFitsView/. 

To read more about Integral Field Spectroscopy, I recommend section 2.2 of my thesis: http://etheses.dur.ac.uk/13311/1/Thesis_cor_mk_II.pdf?DDD25+ or from Chris Harrison; Chapter 2. http://etheses.dur.ac.uk/10744/1/cmh_thesis.pdf?DDD25+


# Dealing wih IFU data

We will first start with loading the cube and extracting a spectrum. We will then continue to write a simple fitting code to fit the spectrum from the previous tutorial. 

Lets start with importing basic libraries. This is a start of every code I write. 

In [ ]:
#importing modules
import numpy as np
import matplotlib.pyplot as plt; plt.ioff()

from astropy.io import fits as pyfits
from astropy import wcs
from astropy.table import Table, join, vstack
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from scipy.optimize import curve_fit

import emcee
import corner 

from os.path import exists
nan= float('nan')

pi= np.pi
e= np.e

plt.close('all')
c= 3.*10**8
h= 6.62*10**-34
k= 1.38*10**-23



PATH='/Users/jansen/My Drive/Astro/'


## Loading the JWST IFS cube. 

We will now load the IFU cube and extract all of the import information. We need to extract the wavelength grid and the data cube and the header information. 

In [ ]:
path = '/Users/jansen/JADES/GA_NIFS/IFS_data_v1/' 
file_name = '5001-GS-3.47-S_jw1216_o005_ff_px0.05_drizzle_ODfde98_3_VSC_MRC_MSA_m2ff_CTX1068.pmap_v1.8.2_g235h-f170lp_cgs_s3d.fits' # Full path to the file

Full_path = path + file_name

with pyfits.open(Full_path, memmap=False) as hdulist:

    Flux_cube = hdulist['SCI'].data *1e4 * 1e15
    Header = hdulist['SCI'].header

    n_channels = Header['NAXIS3'] # Extract the number pixels in the third axis (wavelength)
    obs_wave = Header['CRVAL3'] + (np.arange(n_channels) - (Header['CRPIX3'] - 1.0))*Header['CDELT3'] # Extract the wavelength grid


z = 3.472

plt.show()


## Extracting a single spaxel spectrum

When we looked in QFitsView we could see the individual spaxel spectrum. In python we can to this with simple indexing the right part of the Flux_cube array and also plot it. 

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 5))

x,y  = 43,51 # Coordinates of the spaxel we want to extract the spectrum from
Flux_spaxel = Flux_cube[:, y,x] # Extract the spectrum of the spaxel at position (x,y)
ax.plot( obs_wave, Flux_spaxel, drawstyle='steps-mid')
ax.set_xlabel('wavelength [microns]')
ax.set_ylabel('Flux density - x1e-15 erg/s/micron')



ylow = -0.
yhig = 0.5

ax.vlines(0.5008*(1+z),ylow,yhig, linestyle='dashed',color='orange', alpha=0.8)
ax.vlines(0.6300*(1+z),ylow,yhig, linestyle='dashed',color='orange', alpha=0.8)


ax.vlines(0.6563*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)
ax.vlines(0.4861*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)


ax.vlines(0.6731*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)


plt.show()

We can also zoom in on th Halpha, [NII] and [SII] emission lines, just simply using the ax.set_xlim()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 5))

x,y  = 43,51  # Coordinates of the spaxel we want to extract the spectrum from
Flux_spaxel = Flux_cube[:, y,x] # Extract the spectrum of the spaxel at position (x,y)
ax.plot( obs_wave, Flux_spaxel, drawstyle='steps-mid')
ax.set_xlabel('wavelength [microns]')
ax.set_ylabel('Flux density - x1e-15 erg/s/micron')

ylow = -0.0
yhig = 0.5

ax.vlines(0.5008*(1+z),ylow,yhig, linestyle='dashed',color='orange', alpha=0.8)
ax.vlines(0.6300*(1+z),ylow,yhig, linestyle='dashed',color='orange', alpha=0.8)

ax.vlines(0.6563*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)
ax.vlines(0.4862*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)

ax.vlines(0.6731*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)

ax.set_xlim(2.15,2.3)

plt.show()

or we can look at the [OIII] + Hbeta

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 5))

x,y  = 43,51  # Coordinates of the spaxel we want to extract the spectrum from
Flux_spaxel = Flux_cube[:, y,x] # Extract the spectrum of the spaxel at position (x,y)
ax.plot( obs_wave, Flux_spaxel, drawstyle='steps-mid')
ax.set_xlabel('wavelength [microns]')
ax.set_ylabel('Flux density - x1e-15 erg/s/micron')

ylow = -0.0
yhig = 0.5

ax.vlines(0.5008*(1+z),ylow,yhig, linestyle='dashed',color='orange', alpha=0.8)
ax.vlines(0.6300*(1+z),ylow,yhig, linestyle='dashed',color='orange', alpha=0.8)

ax.vlines(0.6563*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)
ax.vlines(0.4861*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)

ax.vlines(0.6731*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)

ax.set_xlim(2.8,3.1)

plt.show()

## Extracting a spectrum from multiple spaxel together - circular aperture 

The best thing about IFS is that we extract spectrum from anywhere in the galaxy to investigate the properties in that section. We can use a very simple circular mask to setup to create a spectrum with a sum of all of the spaxels. 

In [ ]:
def sp_circ_ap(cenx,ceny,rad,Flux_cube):

    nwl = len(Flux_cube[:,0,0])
    nx = len(Flux_cube[0,0,:])
    ny = len(Flux_cube[0,:,0])
    # creates array with one spectrum per row
    intermarray = np.full((nwl,nx*ny), np.nan)
    # fills it with nans
    
    nloop = 0
    # loop on data
    for x in range(nx) :
        for y in range(ny) :  
            distance = np.sqrt((x-cenx)**2+(y-ceny)**2)
            if distance < rad :
                intermarray[:,int(nloop)] = Flux_cube[:,int(y),int(x)]
                nloop += 1
               
    # average
    outspec = np.nansum(intermarray,axis=1)
    return outspec

# This function extracts the spectrum from a circular aperture of radius 5 pixels centered on the sp
D1_spec = sp_circ_ap(44,47,5,Flux_cube) # 5 pixel radius aperture centered on the spaxel at (44,47)

D1_spec = np.ma.masked_invalid(D1_spec ) # Mask invalid values (nans, infs etc.)

f, ax = plt.subplots(1, 1, figsize=(10, 5))

x,y  = 44,47 # Coordinates of the spaxel we want to extract the spectrum from
ax.plot( obs_wave, D1_spec, drawstyle='steps-mid')
ax.set_xlabel('wavelength [microns]')
ax.set_ylabel('Flux density - x1e-15 erg/s/micron')

ylow = -0.0
yhig = 25

ax.vlines(0.5008*(1+z),ylow,yhig, linestyle='dashed',color='orange', alpha=0.8)
ax.vlines(0.6300*(1+z),ylow,yhig, linestyle='dashed',color='orange', alpha=0.8)

ax.vlines(0.6563*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)
ax.vlines(0.4861*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)

ax.vlines(0.6731*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)
ax.vlines(0.6718*(1+z),ylow,yhig, linestyle='dashed',color='k', alpha=0.5)

ax.set_xlim(2.9,3.03)
ax.set_ylim(-0.,30.)

plt.show()

## Create a emission line map

We can also create a very simple linemap similar to what way we did it in QFitsView. This way we can actually visualize the emission lines and they spatial extent.

In [ ]:
# Creating the linemap:

from QubeSpec import Cube


channel0, channel1 = 3211, 3227 # Values from QFitsView from around Halpha emission line

# Creating a linemap:
Linemap = np.sum(Flux_cube[channel0:channel1, :,:], axis=(0))

# and now plot
f, ax = plt.subplots(1)

ax.imshow(Linemap, cmap='viridis', vmin=0, origin='lower')

plt.show()

## Fitting the spectrum 

We can fit the spectrum this high SNR spectrum based on the skills we have learned in the previous notebook. The setup is going to be very similar but we will fit the Halpha. The Halpha is close to other emission lines - [NII] doublet and [SII] doublet. 

I have pre written part of the function below. However remember that the flues of [NII] doublet is 6585 and 6549 is tied together by a factor of 3 - similar to the [OIII] doublet in the previous notebook. Finally the [SII] doublet ratio is free.  

In [ ]:
# Defining the model for fitting emission lines around Halpha including of course Halpha itself. 
# Setting up a single Gaussian - will be used later 
def gauss(x, k, mu,FWHM):
    sig = FWHM/2.35482/3e5*mu
    expo= -((x-mu)**2)/(2*sig*sig) 
    y= k* e**expo
    return y

# Setting up the Halpha model:
# First we calculate the observed frame wavelengths of the lines
# Then we calculate the individfual components - Halpha, [NII] and [SII]
# Finally we add all of them together
def Halpha_NII_SII(x, z, cont,cont_grad,  Hal_peak, NII_peak, SIIr_peak, SIIb_peak, Nar_fwhm):
    Hal_wv = 6564.52*(1+z)/1e4
    NII_r = 6585.27*(1+z)/1e4
    NII_b = 6549.86*(1+z)/1e4

    SII_r = 6732.67*(1+z)/1e4
    SII_b = 6718.29*(1+z)/1e4

    Hal_nar = gauss(x, Hal_peak, Hal_wv, Nar_fwhm)

    continuum = cont+x*cont_grad

    return continuum+ Hal_nar + NII_r + NII_b + SII_r + SII_b


Next we will prep the data for fitting:
1) will only include channels (data) that was not masked. 

2)  We are not really interested in fitting the whole spectral window. We are only interested in the range our the spectral line. I like to go +- 300 angstroms around the line.

3) Lastly, we need to find the value of the peak of the line - we will use this when creating initial conditions. 

In [ ]:
# Adjust these number to be around Halpha emission line 
fit_loc = np.where((obs_wave>(5008-300)*(1+z)/1e4)&(obs_wave<(5008+300)*(1+z)/1e4))[0] 

sel=  np.where(((obs_wave<(5008+20)*(1+z)/1e4))& (obs_wave>(5008-20)*(1+z)/1e4))[0]
flux_zoom = D1_spec[sel]
wave_zoom = obs_wave[sel]

peak_loc = np.ma.argmax(flux_zoom)
peak = np.ma.max(flux_zoom)

error = np.full_like(D1_spec, np.ma.std(D1_spec[fit_loc])) # Estimate the error as the standard deviation of the flux in the fit region

## Least Square fitting

We are going to first look into least square fitting, using the scipy's curve_fit function - box standard

In [ ]:
from scipy.optimize import curve_fit

# Change initial conditions to match the variables in the function 
init_c = [z,np.median(D1_spec),0.01, peak, peak/4, 400. ]

popt, pcov = curve_fit(Halpha_NII_SII, obs_wave[fit_loc], D1_spec[fit_loc], sigma= error[fit_loc], p0= init_c) 

## Extracting valuable information from the fits

1) Now using the information we used in the 1_Emission_line_tutorial, you can estimate the flux of the different emission lines, their SNRs and FWHM and redshifts. 

2) Also you should fit the OIII+Hbeta emission lines and estimate its fluxes, SNR etc. 

In [ ]:
# Extract Emission line parameters from the fit

In [ ]:
# Fit OIII emission lines